# Importing library

In [72]:
import math
import random
import csv
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [73]:
mydata = pd.read_csv('Example02_Naive_Bayes.csv')
from sklearn.preprocessing import LabelEncoder
enc = LabelEncoder()
enc.fit(mydata['Person'])
mydata['Person'] = enc.transform(mydata['Person'])
new_data=mydata
new_data

,Height,Weight,Foot Size,Person
0,6.00,180,12,1
1,5.92,190,11,1
2,5.58,170,12,1
3,5.92,165,10,1
4,5.00,100,6,0
5,5.50,150,8,0
6,5.42,130,7,0
7,5.75,150,9,0


# the categorical class names are changed to numberic data, Male=0, Female=1

In [74]:
def encode_class(mydata):
    classes = []
    for i in range(len(mydata)):
        if mydata[i][-1] not in classes:
            classes.append(mydata[i][-1])
    for i in range(len(classes)):
        for j in range(len(mydata)):
            if mydata[j][-1] == classes[i]:
                mydata[j][-1] = i
    return mydata

# Splitting the data

In [75]:
def splitting(mydata, ratio):
    train_num = int(len(mydata) * ratio)
    train = []
    # initially testset will have all the dataset
    test = list(mydata)
    while len(train) < train_num:
        # index generated randomly from range 0
        # to length of testset
        index = random.randrange(len(test))
        train.append(test.pop(index))
    return train, test
 

# Group the data rows under each class Male or Female

In [76]:
def groupUnderClass(mydata):
      dict = {}
      for i in range(len(mydata)):
          if (mydata[i][-1] not in dict):
              dict[mydata[i][-1]] = []
          dict[mydata[i][-1]].append(mydata[i])
      return dict
 

# Calculating Mean, Standard Deviation and Create Model

In [77]:
# Calculating Mean
def mean(numbers):
    return sum(numbers) / float(len(numbers))
 
# Calculating Standard Deviation
def std_dev(numbers):
    avg = mean(numbers)
    variance = sum([pow(x - avg, 2) for x in numbers]) / float(len(numbers) - 1)
    return math.sqrt(variance)
 
def MeanAndStdDev(mydata):
    info = [(mean(attribute), std_dev(attribute)) for attribute in zip(*mydata)]
    # eg: list = [ [a, b, c], [m, n, o], [x, y, z]]
    # here mean of 1st attribute =(a + m+x), mean of 2nd attribute = (b + n+y)/3
    # delete summaries of last class
    del info[-1]
    return info
 
# find Mean and Standard Deviation under each class
def MeanAndStdDevForClass(mydata):
    info = {}
    dict = groupUnderClass(mydata)
    for classValue, instances in dict.items():
        info[classValue] = MeanAndStdDev(instances)
    return info
 
 
# Calculate Gaussian Probability Density Function
def calculateGaussianProbability(x, mean, stdev):
    expo = math.exp(-(math.pow(x - mean, 2) / (2 * math.pow(stdev, 2))))
    return (1 / (math.sqrt(2 * math.pi) * stdev)) * expo
 
 
# Calculate Class Probabilities
def calculateClassProbabilities(info, test):
    probabilities = {}
    for classValue, classSummaries in info.items():
        probabilities[classValue] = 1
        for i in range(len(classSummaries)):
            mean, std_dev = classSummaries[i]
            x = test[i]
            probabilities[classValue] *= calculateGaussianProbability(x, mean, std_dev)
    return probabilities
 
 
# Make prediction - highest probability is the prediction
def predict(info, test):
    probabilities = calculateClassProbabilities(info, test)
    bestLabel, bestProb = None, -1
    for classValue, probability in probabilities.items():
        if bestLabel is None or probability > bestProb:
            bestProb = probability
            bestLabel = classValue
    return bestLabel
 
 
# returns predictions for a set of examples
def getPredictions(info, test):
    predictions = []
    for i in range(len(test)):
        result = predict(info, test[i])
        predictions.append(result)
    return predictions
 
# Accuracy score
def accuracy_rate(test, predictions):
    correct = 0
    for i in range(len(test)):
        if test[i][-1] == predictions[i]:
            correct += 1
    return (correct / float(len(test))) * 100.0
 

# Convert Dataset in List

In [113]:
mydata = csv.reader(open('Example02_Naive_Bayes 2.csv'))
mydata = list(mydata)
mydata = encode_class(mydata)

mydata

[['6', '180', '12', 0],
 ['5.92', '190', '11', 0],
 ['5.58', '170', '12', 0],
 ['5.92', '165', '10', 0],
 ['5', '100', '6', 1],
 ['5.5', '150', '8', 1],
 ['5.42', '130', '7', 1],
 ['5.75', '150', '9', 1]]

# Find Accuracy

In [116]:
for i in range(len(mydata)):
    mydata[i] = [float(x) for x in mydata[i]]
     
# split ratio = 0.7
# 70% of data is training data and 30% is test data used for testing
ratio = 0.7
train_data, test_data = splitting(mydata, ratio)
print('Total number of examples are: ', len(mydata))
print('Out of these, training examples are: ', len(train_data))
print("Test examples are: ", len(test_data))
 
# prepare model
info = MeanAndStdDevForClass(train_data)
 
# test model
predictions = getPredictions(info, test_data)
accuracy = accuracy_rate(test_data, predictions)
print("Accuracy of your model is: ", accuracy)

Total number of examples are:  8
Out of these, training examples are:  5
Test examples are:  3
Accuracy of your model is:  66.66666666666666


In [86]:
train_data

[[5.58, 170.0, 12.0, 0.0],
 [6.0, 180.0, 12.0, 0.0],
 [5.0, 100.0, 6.0, 1.0],
 [5.92, 165.0, 10.0, 0.0],
 [5.42, 130.0, 7.0, 1.0]]

In [87]:
test_data

[[5.92, 190.0, 11.0, 0.0], [5.5, 150.0, 8.0, 1.0], [5.75, 150.0, 9.0, 1.0]]

# Prediction 

In [69]:
test_data

[[5.92, 165.0, 10.0, 0.0], [5.5, 150.0, 8.0, 1.0], [5.42, 130.0, 7.0, 1.0]]

In [70]:
predictions

[1.0, 1.0, 1.0]

# Prediction Manually

In [71]:
# 4	5.00	100	6	0
# 5	5.50	150	8	0
# 6	5.42	130	7	0
# 7	5.75	150	9	0

test_data= [[6.0, 180.0, 12.0, 0.0]]
info = MeanAndStdDevForClass(train_data)
predictions = getPredictions(info, test_data)
predictions

[0.0]